In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.optim import Adam, SGD, RMSprop
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [2]:
data = load_breast_cancer()
x = data.data
y = data.target

In [5]:
y[:5].reshape(-1,1)

array([[0],
       [0],
       [0],
       [0],
       [0]])

In [7]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.2,random_state=42)

In [9]:
scal = StandardScaler()
xtrain = scal.fit_transform(xtrain)
xtest = scal.transform(xtest)

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [13]:
xtrain = torch.tensor(xtrain,dtype=torch.float32).to(device)
xtest = torch.tensor(xtest,dtype=torch.float32).to(device)
ytrain = torch.tensor(ytrain,dtype=torch.float32).to(device)
ytest = torch.tensor(ytest,dtype=torch.float32).to(device)

In [15]:
class NeuralNet(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(NeuralNet,self).__init__()
        self.fc1 = nn.Linear(input_size,hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size,output_size)        
        self.act = nn.Sigmoid()

    def forward(self,x):
        out = self.relu(self.fc1(x))
        out = self.relu(out)
        out = self.fc2(out)
        out = self.act(out)
        return out

In [52]:
input_size = xtrain.shape[1]
hidden_size = 64
output_size = 1
lr = 0.0001
epochs = 100

In [38]:
model = NeuralNet(input_size,hidden_size,output_size).to(device)

In [ ]:
out = model(input)
out.view(-1).round().size()

In [54]:
optimizer = Adam(model.parameters(),lr=lr)
criterion = F.binary_cross_entropy#_with_logits

In [60]:
for i in range(epochs):
    model.train()
    optimizer.zero_grad()
    out = model(xtrain)    
    loss = criterion(out,ytrain.view(-1,1))
    loss.backward()
    optimizer.step()
    with torch.no_grad():
        pred = out.round()
        correct = (pred == ytrain.view(-1,1)).sum().item()
        accuracy = 100 * correct/ytrain.shape[0]
    if i % 10 == 0:
        print(f"Epoch {i}/{100} loss : {loss.item():.4f} accuracy : {accuracy:.2f}")

Epoch 0/100 loss : 0.4049 accuracy : 93.85
Epoch 10/100 loss : 0.3950 accuracy : 94.29
Epoch 20/100 loss : 0.3855 accuracy : 94.07
Epoch 30/100 loss : 0.3762 accuracy : 94.07
Epoch 40/100 loss : 0.3672 accuracy : 94.07
Epoch 50/100 loss : 0.3586 accuracy : 94.07
Epoch 60/100 loss : 0.3502 accuracy : 94.07
Epoch 70/100 loss : 0.3421 accuracy : 94.07
Epoch 80/100 loss : 0.3342 accuracy : 94.07
Epoch 90/100 loss : 0.3266 accuracy : 94.51


In [62]:
pred = model(xtest)
pred = pred.round()
correct = (pred == ytest.view(-1,1)).sum().item()
accuracy = 100 * correct/ytest.shape[0]
print(f"correct : {correct}/{ytest.shape[0]} , accuracy : {accuracy:.2f}")

correct : 111/114 , accuracy : 97.37


In [49]:
index = 15
out = model(xtest[index])
original = ytest[index]
print(f"Predicted : {out[0]:.1f}")
print(f"Original : {original}")

Predicted : 0.0
Original : 0.0
